In [ ]:
import os
import logging
import random 
import tqdm

import adios2 as ad2
#import pymetis

import torch
import numpy as np
import random

from sklearn.model_selection import train_test_split


In [ ]:
#!pip freeze | grep adios2
conda install -c conda-forge pymetis 

In [ ]:
conda install pytorch-model-summary

In [ ]:
conda install adios2

In [ ]:
def read_f0(istep, expdir=None, iphi=None, inode=0, nnodes=None, average=False, 
            randomread=0.0, nchunk=16, fieldline=False):
    """
    Read XGC f0 data
    """
    def adios2_get_shape(f, varname):
        nstep = int(f.available_variables()[varname]['AvailableStepsCount'])
        shape = f.available_variables()[varname]['Shape']
        lshape = None
        if shape == '':
            ## Accessing Adios1 file
            ## Read data and figure out
            v = f.read(varname)
            lshape = v.shape
        else:
            lshape = tuple([ int(x.strip(',')) for x in shape.strip().split() ])
        return (nstep, lshape)

    fname = os.path.join(expdir, 'restart_dir/xgc.f0.%05d.bp'%istep)
    if randomread > 0.0:
        ## prefetch to get metadata
        with ad2.open(fname, 'r') as f:
            nstep, nsize = adios2_get_shape(f, 'i_f')
            ndim = len(nsize)
            nphi = nsize[0]
            _nnodes = nsize[2] if nnodes is None else nnodes
            nmu = nsize[1]
            nvp = nsize[3]
        assert _nnodes%nchunk == 0
        _lnodes = list(range(inode, inode+_nnodes, nchunk))
        lnodes = random.sample(_lnodes, k=int(len(_lnodes)*randomread))
        lnodes = np.sort(lnodes)

        lf = list()
        li = list()
        for i in tqdm(lnodes):
            li.append(np.array(range(i,i+nchunk), dtype=np.int32))
            with ad2.open(fname, 'r') as f:
                nphi = nsize[0] if iphi is None else 1
                iphi = 0 if iphi is None else iphi 
                #this is considering single planes, there are 8 planes
                start = (iphi,0,i,0)
                count = (nphi,nmu,nchunk,nvp)
                _f = f.read('i_f', start=start, count=count).astype('float64')
                lf.append(_f)
        i_f = np.concatenate(lf, axis=2)
        lb = np.concatenate(li)
    elif fieldline is True:
        import networkx as nx

        fname2 = os.path.join(expdir, 'xgc.mesh.bp')
        with ad2.open(fname2, 'r') as f:
            _nnodes = int(f.read('n_n', ))
            nextnode = f.read('nextnode')
        
        G = nx.Graph()
        for i in range(_nnodes):
            G.add_node(i)
        for i in range(_nnodes):
            G.add_edge(i, nextnode[i])
            G.add_edge(nextnode[i], i)
        cc = [x for x in list(nx.connected_components(G)) if len(x) >= 16]

        li = list()
        for k, components in enumerate(cc):
            DG = nx.DiGraph()
            for i in components:
                DG.add_node(i)
            for i in components:
                DG.add_edge(i, nextnode[i])
            
            cycle = list(nx.find_cycle(DG))
            DG.remove_edge(*cycle[-1])
            
            path = nx.dag_longest_path(DG)
            #print (k, len(components), path[0])
            for i in path[:len(path)-len(path)%16]:
                li.append(i)

        with ad2.open(fname, 'r') as f:
            nstep, nsize = adios2_get_shape(f, 'i_f')
            ndim = len(nsize)
            nphi = nsize[0] if iphi is None else 1
            iphi = 0 if iphi is None else iphi
            _nnodes = nsize[2]
            nmu = nsize[1]
            nvp = nsize[3]
            start = (iphi,0,0,0)
            count = (nphi,nmu,_nnodes,nvp)
            logging.info (f"Reading: {start} {count}")
            i_f = f.read('i_f', start=start, count=count).astype('float64')
        
        _nnodes = len(li)-inode if nnodes is None else nnodes
        lb = np.array(li[inode:inode+_nnodes], dtype=np.int32)
        logging.info (f"Fieldline: {len(lb)}")
        logging.info (f"{lb}")
        i_f = i_f[:,:,lb,:]
    else:
        with ad2.open(fname, 'r') as f:
            nstep, nsize = adios2_get_shape(f, 'i_f')
            ndim = len(nsize)
            nphi = nsize[0] if iphi is None else 1
            iphi = 0 if iphi is None else iphi
            _nnodes = nsize[2]-inode if nnodes is None else nnodes
            nmu = nsize[1]
            nvp = nsize[3]
            start = (iphi,0,inode,0)
            count = (nphi,nmu,_nnodes,nvp)
            logging.info (f"Reading: {start} {count}")
            i_f = f.read('i_f', start=start, count=count).astype('float64')
            #e_f = f.read('e_f')
        li = list(range(inode, inode+_nnodes))
        lb = np.array(li, dtype=np.int32)

    if i_f.shape[3] == 39:
        i_f = np.append(i_f, i_f[...,38:39], axis=3)
        i_f = np.append(i_f, i_f[:,38:39,:,:], axis=1)

    Z0 = np.moveaxis(i_f, 1, 2)

    if average:
        Z0 = np.mean(Z0, axis=0)
        zlb = lb
    else:
        Z0 = Z0.reshape((-1,Z0.shape[2],Z0.shape[3]))
        _lb = list()
        for i in range(nphi):
            _lb.append( i*100_000_000 + lb)
        zlb = np.concatenate(_lb)
    
    #zlb = np.concatenate(li)
    zmu = np.mean(Z0, axis=(1,2))
    zsig = np.std(Z0, axis=(1,2))
    zmin = np.min(Z0, axis=(1,2))
    zmax = np.max(Z0, axis=(1,2))
    Zif = (Z0 - zmin[:,np.newaxis,np.newaxis])/(zmax-zmin)[:,np.newaxis,np.newaxis]

    return (Z0, Zif, zmu, zsig, zmin, zmax, zlb)

def read_data(base_data_dir, super_data_dir, num_channels=1):
    Z0, Zif, zmu, zsig, zmin, zmax, zlb = read_f0(420, expdir=base_data_dir, iphi=0)
    Z0_s, Zif_s, zmu_s, zsig_s, zmin_s, zmax_s, zlb_s = read_f0(420, expdir=super_data_dir, iphi=0)
    #print('base:',Zif.shape, zlb.shape, zmu.shape, zsig.shape)
    #print('super:',Zif_s.shape, zlb_s.shape, zmu_s.shape, zsig_s.shape)
    
    lx = list()
    ly = list()
    for i in range(0,len(Zif)-num_channels,num_channels):
        X = Zif[i:i+num_channels,:,:]
        lx.append(X)
        ly.append(zlb[i:i+num_channels])
    
    X_train, X_test, id_train, id_test = train_test_split(lx, ly, test_size=0.10, random_state=42)
    
    #Y_train = list()
    #Y_test = list()
    x_s = list()
    for i in range(0,len(Zif_s)-num_channels,num_channels):
        X = Zif_s[i:i+num_channels,:,:]
        x_s.append(X)
    
    '''
    for ids in id_train:
        X= Zif_s[ids[0]:ids[0]+num_channels,:,:]
        Y_train.append(X)
    
    for ids in id_test:
        X= Zif_s[ids[0]:ids[0]+num_channels,:,:]
        Y_test.append(X)
    '''
    
    return lx,x_s, id_train, id_test


In [ ]:
class XGCSuperDatasetMultiRegion:
    def __init__(self, base_X, base_Y, ids, num_cluster, node_per_anchor, patch_size):
        
        super().__init__()
        self.image_X = base_X
        self.image_Y = base_Y
        self.id_list = ids
        self.num_grp=num_cluster
        self.patch_size = patch_size
        self.grp=[]
        self.id_len=len(node_per_anchor[0])
        for c in range(num_cluster):
            self.grp.append(node_per_anchor[c])
        
        orig_sz = base_X[0].shape[1]
        self.image_size = int(orig_sz/patch_size)
        self.num_patches = int((orig_sz*orig_sz)/(patch_size*patch_size))
        self.ids =[]
        self.sub_ids=[]
        
        for i in range(0,self.id_len):
            self.ids+=self.num_patches*[i]
            self.sub_ids+=range(0,self.num_patches)
    
        #print('init:',len(self.ids),self.grp[0][self.ids[0]],self.grp[1][self.ids[0]])
    
    def __len__(self): 
        return len(self.ids) 
    
    def __getitem__(self,i):
        idx = self.ids[i]
        sub_idx = self.sub_ids[i]
        
        ridx = int(sub_idx/self.image_size)
        cidx = int(sub_idx%self.image_size)
        
        rs= ridx*self.patch_size
        re = rs+self.patch_size
        cs = cidx*self.patch_size
        ce = cs+self.patch_size
        
        base_images=[]
        super_images=[]
        labels=[]
        si_list =[]
        for j in range(self.num_grp):
            label = self.grp[j][idx]
            base_image = self.image_X[label]
            base_image=base_image[0,:,:]
        
            super_image = self.image_Y[label]
            super_image= super_image[0,:,:]
            
            X_img = base_image[rs:re,cs:ce]
            X_img = X_img[np.newaxis,:,:]
        
            Y_img = super_image[rs:re,cs:ce]
            Y_img = Y_img[np.newaxis,:,:]
            
            si = np.zeros(self.num_grp)
            si[j] = 1
            
            base_images.append(X_img)
            super_images.append(Y_img)
            labels.append(label)
            si_list.append(si)
        
        #print(orig_image.shape)
        sample1 = {'X': torch.as_tensor(base_images[0].copy()).float(), 
                  'Y': torch.as_tensor(super_images[0].copy()),
                  'S': torch.as_tensor(si_list[0].copy()),
                  'label': labels[0],
                  'rsid': rs, 'csid':cs}
        
        sample2 = {'X': torch.as_tensor(base_images[1].copy()).float(), 
                  'Y': torch.as_tensor(super_images[1].copy()),
                  'S': torch.as_tensor(si_list[1].copy()),
                  'label': labels[1],
                  'rsid': rs, 'csid':cs}
        
        sample3 = {'X': torch.as_tensor(base_images[2].copy()).float(), 
                  'Y': torch.as_tensor(super_images[2].copy()),
                  'S': torch.as_tensor(si_list[2].copy()),
                  'label': labels[2],
                  'rsid': rs, 'csid':cs}
        
        sample4 = {'X': torch.as_tensor(base_images[3].copy()).float(), 
                  'Y': torch.as_tensor(super_images[3].copy()),
                  'S': torch.as_tensor(si_list[3].copy()),
                  'label': labels[3],
                  'rsid': rs, 'csid':cs}
        
        sample5 = {'X': torch.as_tensor(base_images[4].copy()).float(), 
                  'Y': torch.as_tensor(super_images[4].copy()),
                  'S': torch.as_tensor(si_list[4].copy()),
                  'label': labels[4],
                  'rsid': rs, 'csid':cs}
        
        
        return [sample1,sample2,sample3,sample4,sample5]

# Select anchor nodes over mesh

In [ ]:
#"/Users/oit/desktop/anika-macbook/datasets/ornl/fusion-energy-xgc/d3d_coarse_v2_colab"
inputdir="/gpfs/alpine/world-shared/csc143/jyc/summit"
with ad2.open('{}/d3d_coarse_small_v2/xgc.mesh.bp'.format(inputdir), 'r') as f:
    nnodes = int(f.read('n_n', ))
    ncells = int(f.read('n_t', ))
    rz = f.read('rz')
    conn = f.read('nd_connect_list')
    psi = f.read('psi')
    
r = rz[:,0]
z = rz[:,1]
sml_nphi = 16

In [ ]:
conn.shape,psi.shape,r.shape,z.shape,nnodes,ncells

In [ ]:
def spatial_cluster(nnodes,r,z,num_anchor=5):
    pairwise_dis=np.zeros(nnodes)
    #vec_0 = np.sqrt((r[0]**2+z[0]**2))
    
    for i in range(1, nnodes):
        p = (r[i]-r[0])**2 + (z[i]-z[0])**2
        pairwise_dis[i] = np.sqrt(p)
    
    #print(np.min(pairwise_dis), np.max(pairwise_dis), np.mean(pairwise_dis[1:]))
    k = num_anchor
    nodes_per_cluster = int(nnodes/k)
    node_set_per_anchor={}
    cluster_per_node={}
    
    for i in range(k):
        node_set_per_anchor[i]=[]
    
    for i in range(nnodes):
        if i<=nodes_per_cluster:
            cluster_per_node[i] =0
            node_set_per_anchor[0].append(i)
        elif i>=nodes_per_cluster and i<=2*nodes_per_cluster:
            cluster_per_node[i] =1
            node_set_per_anchor[1].append(i)
        elif i>2*nodes_per_cluster and i<= 3*nodes_per_cluster:
            cluster_per_node[i] =2
            node_set_per_anchor[2].append(i)
        elif i>3*nodes_per_cluster and i<=4*nodes_per_cluster:
            cluster_per_node[i] =3
            node_set_per_anchor[3].append(i)
        elif i>4*nodes_per_cluster:
            cluster_per_node[i] =4
            node_set_per_anchor[4].append(i)


    return pairwise_dis, cluster_per_node, node_set_per_anchor


Hyper Network Module

In [ ]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F

class HyperNetwork(nn.Module):

    def __init__(self, f_size = 3, z_dim = 64, out_size=16, in_size=16,s_dim=100):
        super(HyperNetwork, self).__init__()
        self.z_dim = z_dim
        self.f_size = f_size
        self.out_size = out_size
        self.in_size = in_size
        self.s_dim = s_dim
        
        #64 X 16*3*3
        self.w1 = Parameter(torch.fmod(torch.randn((self.z_dim, self.out_size*self.f_size*self.f_size)).cuda(),2))
        #16*3*3
        self.b1 = Parameter(torch.fmod(torch.randn((self.out_size*self.f_size*self.f_size)).cuda(),2))
        
        #64 X 16*64
        self.w2 = Parameter(torch.fmod(torch.randn((self.z_dim, self.in_size*self.z_dim)).cuda(),2))
        #16*64 X 1
        self.b2 = Parameter(torch.fmod(torch.randn((self.in_size*self.z_dim)).cuda(),2))
        
        #100 X 16*64
        self.w3 = Parameter(torch.fmod(torch.randn((self.s_dim, self.in_size*self.z_dim)).cuda(),2))
        #16*64 X 1
        self.b3 = Parameter(torch.fmod(torch.randn((self.in_size*self.z_dim)).cuda(),2))
        
    def forward(self, z,si): #100 

        h_in = torch.matmul(z, self.w2) + self.b2    # input 64 dim , output = 16*64
        h_in = h_in.view(self.in_size, self.z_dim)   #16 X 64
        
        h_spatial_in = torch.matmul(si,self.w3) + self.b3  #input 100 dim, output = 16*64 
        h_spatial_in = h_spatial_in.view(self.in_size, self.z_dim) #16 X 64
        
        h_new = h_spatial_in + h_in #Add values of learnable and spatial embeddings once in same latent space

        #h_final = torch.matmul(h_in, self.w1) + self.b1  # input 16 X 64, output = 16 X 16*3*3 
        #kernel = h_final.view(self.out_size, self.in_size, self.f_size, self.f_size) # 16 X 16 X 3 X 3
        
        h_final = torch.matmul(h_new, self.w1) + self.b1  # input 16 X 64, output = 16 X 16*3*3 
        kernel = h_final.view(self.out_size, self.in_size, self.f_size, self.f_size) # 16 X 16 X 3 X 3

        return kernel







In [ ]:
#ResNet Block

class IdentityLayer(nn.Module):

    def forward(self, x):
        return x


class ResNetBlock(nn.Module):

    def __init__(self, in_size=16, out_size=16, downsample = False):
        super(ResNetBlock,self).__init__()
        self.out_size = out_size
        self.in_size = in_size
        if downsample:
            self.stride1 = 2
            self.reslayer = nn.Conv2d(in_channels=self.in_size, out_channels=self.out_size, 
                                      stride=2, kernel_size=1)
        else:
            self.stride1 = 1
            self.reslayer = IdentityLayer()

        self.bn1 = nn.BatchNorm2d(out_size)
        self.bn2 = nn.BatchNorm2d(out_size)

    def forward(self, x, conv1_w, conv2_w):

        residual = self.reslayer(x)

        out = F.relu(self.bn1(F.conv2d(x, conv1_w, stride=self.stride1, padding=1)), inplace=True)
        out = self.bn2(F.conv2d(out, conv2_w, padding=1))

        out += residual

        out = F.relu(out)

        return out


In [ ]:

class Embedding(nn.Module):

    def __init__(self, z_num, z_dim):
        super(Embedding, self).__init__()

        self.z_list = nn.ParameterList()
        self.z_num = z_num
        self.z_dim = z_dim

        h,k = self.z_num

        for i in range(h):
            for j in range(k):
                self.z_list.append(Parameter(torch.fmod(torch.randn(self.z_dim).cuda(), 2)))

    def forward(self, hyper_net,si): #si is the spatial encoding vector.
        ww = []
        h, k = self.z_num
        for i in range(h):
            w = []
            for j in range(k):
                w.append(hyper_net(self.z_list[i*k + j],si))
            ww.append(torch.cat(w, dim=1))
        return torch.cat(ww, dim=0)


class PrimaryNetwork(nn.Module):

    def __init__(self, z_dim=64, patch_size=5, s_dim=99):
        super(PrimaryNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.patch_size= patch_size
        self.z_dim = z_dim
        self.s_dim = s_dim
        self.hope = HyperNetwork(z_dim=self.z_dim, s_dim = self.s_dim)

        self.zs_size = [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1],
                        [2, 1], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2],
                        [4, 2], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4]]

        self.filter_size = [[16,16], [16,16], [16,16], [16,16], [16,16], [16,16], [16,32], [32,32], [32,32], [32,32],
                            [32,32], [32,32], [32,64], [64,64], [64,64], [64,64], [64,64], [64,64]]

        self.res_net = nn.ModuleList()

        for i in range(18):
            down_sample = False
            if i > 5 and i % 6 == 0:
                down_sample = True
            self.res_net.append(ResNetBlock(self.filter_size[i][0], self.filter_size[i][1], downsample=down_sample))

        self.zs = nn.ModuleList()

        for i in range(36):
            self.zs.append(Embedding(self.zs_size[i], self.z_dim))

        self.global_avg = nn.AvgPool2d(8)
        self.final = nn.Linear(256,self.patch_size*self.patch_size)

    def forward(self, x,si): # where si is the (num_nodes X 1) spatial-encoding vector

        x = F.relu(self.bn1(self.conv1(x)))
        #print('conv1:',x.shape)
        for i in range(18):
            # if i != 15 and i != 17:
            w1 = self.zs[2*i](self.hope,si) #Note the SAME si is passed for all sub_parts x of a single image X_i
            w2 = self.zs[2*i+1](self.hope,si)
            x = self.res_net[i](x, w1, w2)
            #print('resnet:',i,x.shape)
        
        #print('final resnet:',x.shape)
        
        #x = self.global_avg(x)
        #print('avg pool:',x.shape)
        x = self.final(x.view(-1,256))
        x = x.view(-1,1,self.patch_size,self.patch_size)

        return x


In [ ]:
from pytorch_model_summary import summary
net = PrimaryNetwork(patch_size=8, s_dim=5)
total=0
for name, param in net.named_parameters():
    if param.requires_grad:
        if name.find('res_net'):
            #print(name, param.numel())
            total+=param.numel()
        else:
            continue      

print('total:',total)

Train Model

In [ ]:
import os
from torch.autograd import Variable
import time

import torch.optim as optim


In [ ]:
def set_data(patch_size, batch_size, num_anchor):
    
    X, Y, id_train, id_test = read_data(dir_base_data, dir_super_data)
    
    si, cluster_per_node, node_per_anchor = spatial_cluster(nnodes, r, z, num_anchor=num_anchor)
    max_id_train= 0
    max_id_test = 0
    
    #print(len(X_train), len(X_test), len(Y_train), len(Y_test))
    train_node_per_anchor={}
    test_node_per_anchor={}
    for idx in range(num_anchor):
        list_ids = node_per_anchor[idx]
        list_train =[]
        list_test =[]
        for ids in list_ids:
            if ids in id_train:
                list_train.append(ids)
            elif ids in id_test:
                list_test.append(ids)
        
        train_node_per_anchor[idx] = list_train
        test_node_per_anchor[idx] = list_test
        
        if len(list_train)>max_id_train:
            max_id_train = len(list_train)
        
        if len(list_test)>max_id_test:
            max_id_test = len(list_test)
    
    #print('max #nodes:',max_id_train, max_id_test)
    for idx in range(num_anchor):
        list_ids_tr =  train_node_per_anchor[idx]
        list_ids_te = test_node_per_anchor[idx]
        
        if len(list_ids_tr)<max_id_train:
            num_xtra_samples = max_id_train-len(list_ids_tr)
            samples = random.sample(list_ids_tr,num_xtra_samples)
            list_ids_tr+=samples
            train_node_per_anchor[idx] = list_ids_tr
        
        if len(list_ids_te)<max_id_test:
            num_xtra_samples = max_id_test-len(list_ids_te)
            samples = random.sample(list_ids_te,num_xtra_samples)
            list_ids_te+=samples
            test_node_per_anchor[idx] = list_ids_te
        
        #print('idx:',idx, len(train_node_per_anchor[idx]),len(test_node_per_anchor[idx]))
        
    trainset = XGCSuperDatasetMultiRegion(X, Y, id_train, num_anchor, train_node_per_anchor, patch_size)
    
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False, num_workers=0)
        
    testset = XGCSuperDatasetMultiRegion(X, Y, id_test, num_anchor, test_node_per_anchor, patch_size)
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    print('data loader:',idx, len(trainloader),len(testloader))
    
    return trainloader, testloader



In [ ]:
def validate(device, net, testloader, criterion, num_cluster):
    correct = 0.
    total =0 
    num_test=0
    #net.eval()
    for tdata in testloader:
        for cl in range(num_cluster):
            timages, tlabels, S = tdata[cl]['X'], tdata[cl]['Y'], tdata[cl]['S']
            #tlables = Variable(tlabels1.cuda())
            timages = Variable(timages.cuda())
            si = Variable(S[0].cuda())
            with torch.no_grad():   
                toutputs = net(timages.double(),si.double())
                predicted = toutputs.cpu().data
                error= criterion(predicted,tlabels)
                correct+=error.item()
        
        total+=num_cluster
        
    return correct, total


In [ ]:
epochs = 50
batch_size = 50 #256
save_freq = 20
patch_size = 8 
print_freq = 20
s_dim =5
dir_out = 'checkpoint/hypernet-concentric-microbatch/'
dir_resume = 'checkpoint/hypernet-concentric-microbatch/hypernet_plasma_35.pth'
resume_epoch = 35
dir_base_data = '/gpfs/alpine/world-shared/csc143/jyc/summit/d3d_coarse_small_v2' 
dir_super_data = '/gpfs/alpine/world-shared/csc143/jyc/summit/d3d_coarse_small_v2_4x' 
resume = True


if not os.path.exists(dir_out):
        os.makedirs(dir_out)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training Model

In [ ]:
def train_microbatch(device, s_dim):
    trainloaders, testloaders = set_data(patch_size, batch_size, s_dim)
    
    net = PrimaryNetwork(patch_size=patch_size, s_dim = s_dim)
    best_accuracy = 10000.
    total_iter = 0
    epoch = 0
    loss_file=open(dir_out+'microbatch_loss.txt','a')
    if resume:
        ckpt = torch.load(dir_resume)
        net.load_state_dict(ckpt['net'])
        best_accuracy = ckpt['acc']
        total_iter= resume_epoch
        epoch=resume_epoch
    else:
        loss_file.write('epoch,loss1,loss2,loss3,loss4,loss5,loss,val_loss\n')
    net = net.double()
    net.cuda()

    learning_rate = 0.0001
    weight_decay = 0.0005
    milestones = [100, 150, 200, 500, 1000]
    max_iter = epochs

    optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
    #lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=milestones, gamma=0.5)
    criterion = nn.MSELoss()

    
    num_batch = len(trainloaders)
    #print_freq = args.print_freq
    
    
    start = time.time()
    
    while total_iter < max_iter:

        running_loss = 0.0
        epoch_loss=0
        grp_loss=np.zeros(s_dim)
        epoch=0
        optimizer.zero_grad()
        
        for i, sample in enumerate(trainloaders,0):
            optimizer.zero_grad()
            for cl in range(s_dim-1):
                #Sample 1
                inputs = sample[cl]['X']
                labels = sample[cl]['Y']
                si = sample[cl]['S']
                
                inputs= Variable(inputs.cuda())
                labels = Variable(labels.cuda())
                si = Variable(si[0].cuda())
                
                outputs = net(inputs.double(), si.double())
                loss = criterion(outputs, labels) #+ physics_loss()
                loss.backward(retain_graph=True)
                
                grp_loss[cl]+=loss.item()
                
                running_loss += loss.item()
                epoch_loss+=loss.item()
            
            #last Sample of the microbatch
            cl = s_dim-1
            inputs_l = sample[cl]['X']
            labels_l = sample[cl]['Y']
            si_l = sample[cl]['S']
            
            inputs_l= Variable(inputs_l.cuda())
            labels_l = Variable(labels_l.cuda())
            si_l = Variable(si_l[0].cuda())
            
            outputs_l = net(inputs_l.double(), si_l.double())
            
            loss = criterion(outputs_l, labels_l) #+ physics_loss()
            loss.backward()
            
            optimizer.step()
            #lr_scheduler.step()
            
            grp_loss[cl]+=loss.item()
            running_loss += loss.item()
            epoch_loss+=loss.item()
            if i % print_freq == (print_freq-1):
                print("[Epoch %d, loader %d] Loss: %.6f" % (total_iter + 1, 
                                                                i + 1, 
                                                                running_loss/(s_dim*print_freq)))
            running_loss = 0.0

        
        epoch += 1
        
        total_iter += 1
        
        correct, total= validate(device, net, testloaders, criterion, s_dim) #Update this function appropriately.
        
        epoch_loss/=(num_batch*s_dim)
        grp_loss/=num_batch
        val_loss = correct/total
        string2=''
        for cl in range(s_dim):
            string2+= str(grp_loss[cl])+','
        
        string1=str(total_iter)+','+string2+str(epoch_loss)+','+str(val_loss)+'\n'
        loss_file.write(string1)
        print('After epoch %d, validation loss: %.6f' % (total_iter, val_loss))

        if val_loss < best_accuracy:
            print('Saving model...')
            state = {
                'net': net.state_dict(),
                'acc': val_loss
            }
            torch.save(state, dir_out+'hypernet_plasma_'+str(total_iter)+'.pth')
            best_accuracy = val_loss
        
    print('Finished Training')
    state = {
                'net': net.state_dict(),
                'acc': val_loss
            }
    torch.save(state, dir_out+'last.pth')
    loss_file.close()
    
    

In [ ]:
import time
start = time.time()
train_microbatch(device, s_dim)
end = time.time()

total = end-start
print('Finished Training in '+str(total)+' sec.')   

Test Data

In [ ]:
def load_data(batch_size, dir_base_data, dir_super_data, patch_size, num_anchor, readall=False):
    if readall:
        X, Y, id_test = read_all(dir_base_data, dir_super_data)
    else:
        X, Y, id_train, id_test = read_data(dir_base_data, dir_super_data)
    
    si, cluster_per_node, node_per_anchor = spatial_cluster(nnodes, r, z, num_anchor=num_anchor)
    
    max_id_test = 0
    
    test_node_per_anchor={}
    for idx in range(num_anchor):
        list_ids = node_per_anchor[idx]
        list_test =[]
        for ids in list_ids:
            if ids in id_test:
                list_test.append(ids)
        test_node_per_anchor[idx] = list_test
        
        if len(list_test)>max_id_test:
            max_id_test = len(list_test)
    
    #print('max #nodes:',max_id_train, max_id_test)
    for idx in range(num_anchor):
        list_ids_te = test_node_per_anchor[idx]
        
        if len(list_ids_te)<max_id_test:
            num_xtra_samples = max_id_test-len(list_ids_te)
            samples = random.sample(list_ids_te,num_xtra_samples)
            list_ids_te+=samples
            test_node_per_anchor[idx] = list_ids_te
        
    
    testset = XGCSuperDatasetMultiRegion(X, Y, list_test, num_anchor, test_node_per_anchor, patch_size)
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
        
        
    return testloader, X, Y, id_test



In [ ]:
def load_model(device, dir_model, s_dim):
    net = PrimaryNetwork(patch_size=patch_size, s_dim=s_dim)
    ckpt = torch.load(dir_model, map_location=device)
    state_dict = ckpt['net']
    
    new_state_dict = {}
    
    for k, v in state_dict.items():
        k = k.replace("module.", "")
        new_state_dict[k] = v
        state_dict = new_state_dict
    
    net.to(device=device)
    net.load_state_dict(state_dict)
    
    return net


In [ ]:
def aggregate(map_pred_img,predictions, rid, cid, labels, patch_size):
    
    for l in range(0,len(labels)):
        #print(labels[l])
        map_pred_img[labels[l]][rid[l]:rid[l]+patch_size,cid[l]:cid[l]+patch_size] = predictions[l,:,:]
        
    return map_pred_img


In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
def test(device, batch_size, patch_size, dir_out, dir_model, dir_base_data, dir_super_data, num_anchor):
    model = load_model(device, dir_model, num_anchor)
    model = model.double()
    testloaders, Xtest, Ytest, id_test = load_data(batch_size, 
                                    dir_base_data, dir_super_data, patch_size, num_anchor)
    model.eval()
    map_pred_img_ensemble={}
    
    for tid in id_test:
        #print('test labels:',tid[0])
        map_pred_img_ensemble[tid[0]]= np.zeros((40,40))
        
    for i, data in enumerate(testloaders,0):
        for cl in range(num_anchor):
            timages, tlabels, S = data[cl]['X'], data[cl]['Y'], data[cl]['S']
            nid, rid, cid = data[cl]['label'],data[cl]['rsid'], data[cl]['csid']
            #tlables = Variable(tlabels1.cuda())
            timages = Variable(timages.cuda())
            si = Variable(S[0].cuda())
            with torch.no_grad():   
                toutputs = model(timages.double(),si.double())
                predicted = toutputs.cpu().data
                
            predicted = predicted.squeeze()
            predicted = predicted.numpy()
            tids = list(nid.numpy())
            rid = list(rid.numpy())
            cid = list(cid.numpy())
            #print(i,predicted.shape,orig_image.shape,len(rid),len(cid),len(tlabels))
        
            map_pred_img_ensemble = aggregate(map_pred_img_ensemble,predicted, rid, cid, 
                                          tids, patch_size)
    
    fname = 'rmse_xgc_test_'+str(batch_size)+'_'+str(len(testloaders))+'.txt'
    error_file=open(dir_out+fname,'w')
    total_err =0
    Ypred =[]
    for l in range(0,len(id_test)):
        tid = id_test[l][0]
        targets = map_pred_img_ensemble[tid]
        loss = rmse(targets, Ytest[l][0])
        total_err+=loss
        #print(tid,loss,Xtest[l][0].shape, targets.shape)
        string=str(tid)+','+str(loss.item())+'\n'
        error_file.write(string)
        Ypred.append(targets)
        
    print('total:',total_err/len(id_test))
    string='total,'+str(total_err)+'\n'
    error_file.write(string)
        
    error_file.close()
    
    return Xtest, Ytest, Ypred, id_test


In [ ]:
batch_size = 50
patch_size = 8
s_dim = 5
dir_out = 'results/hypernet-concentric-microbatch/'
dir_model = 'checkpoint/hypernet-concentric-microbatch/hypernet_plasma_50.pth'
dir_base_data = '/gpfs/alpine/world-shared/csc143/jyc/summit/d3d_coarse_small_v2'
dir_super_data = '/gpfs/alpine/world-shared/csc143/jyc/summit/d3d_coarse_small_v2_4x'
if not os.path.exists(dir_out):
        os.makedirs(dir_out)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xtest, ytest, ypred, idtest = test(device,batch_size, patch_size, dir_out, dir_model,
                                   dir_base_data, dir_super_data, s_dim)

In [ ]:
for l in range(0,len(idtest)):
    np.save(dir_out+'f_pred_'+str(idtest[l][0]),ypred[l])


In [ ]:
import matplotlib.pyplot as plt

_, ny, nx = ytest[0].shape
ix = np.linspace(0, nx-1, nx)
iy = np.linspace(0, ny-1, ny)
Mx, My = np.meshgrid(ix, iy)

for l in range(0,len(ytest)):
    #plt.figure(figsize=(4, 9))
    print('idx:',l)
    f = plt.figure(figsize=(12, 8))
    f.add_subplot(1,3, 1)
    plt.imshow(xtest[l][0], origin='lower')
    #plt.colorbar()
    plt.contour(Mx, My, ytest[l][0], levels=5, origin='image', colors='white', alpha=0.5)
    plt.axis('scaled')
    plt.axis('off')
    plt.title('input %d'%(idtest[l][0]))
    plt.tight_layout()
    
    f.add_subplot(1,3, 2)
    plt.imshow(ytest[l][0], origin='lower')
    #plt.colorbar()
    plt.contour(Mx, My, ytest[l][0], levels=5, origin='image', colors='white', alpha=0.5)
    plt.axis('scaled')
    plt.axis('off')
    plt.title('original %d'%(idtest[l][0]))
    plt.tight_layout()
    
    f.add_subplot(1,3, 3)
    plt.imshow(ypred[l], origin='lower')
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.contour(Mx, My, ypred[l], levels=5, origin='image', colors='white', alpha=0.5)
    #RMSE = rmse(ypred[l], ytest[l][0])
    #plt.text(.02,.95,'RMSE: {:.04f}'.format(RMSE), fontsize=14, c='white')
    plt.axis('scaled')
    plt.axis('off')
    plt.title('predicted %d'%(idtest[l][0]))
    plt.tight_layout()
    plt.show()
    
    if l>100:
        break


# Plot Error across Mesh

In [ ]:
def read_all(base_data_dir, super_data_dir, num_channels=1):
    Z0, Zif, zmu, zsig, zmin, zmax, zlb = read_f0(420, expdir=base_data_dir, iphi=0)
    Z0_s, Zif_s, zmu_s, zsig_s, zmin_s, zmax_s, zlb_s = read_f0(420, expdir=super_data_dir, iphi=0)
    #print('base:',Zif.shape, zlb.shape, zmu.shape, zsig.shape)
    #print('super:',Zif_s.shape, zlb_s.shape, zmu_s.shape, zsig_s.shape)
    
    lx = list()
    ly = list()
    lid = list()
    for i in range(0,len(Zif),num_channels):
        X = Zif[i:i+num_channels,:,:]
        lx.append(X)
        lid.append(zlb[i:i+num_channels])
    
    #print('lid:',lid[:10])
    for ids in lid:
        X= Zif_s[ids[0]:ids[0]+num_channels,:,:]
        ly.append(X)
    
    
    return lx, ly, lid


In [ ]:
def test_all(device, batch_size, patch_size, dir_out, dir_model, dir_base_data, dir_super_data, s_dim):
    model = load_model(device, dir_model, s_dim)
    model = model.double()
    testloaders, Xtest, Ytest, id_test= load_data(batch_size, dir_base_data, 
                                                dir_super_data, patch_size, s_dim, 
                                                  readall=True)
    model.eval()
    map_pred_img_ensemble={}
    
    for tid in id_test:
        #print('test labels:',tid[0])
        map_pred_img_ensemble[tid[0]]= np.zeros((40,40))
    
    
    for i, data in enumerate(testloaders,0):
        for cl in range(s_dim):
            timages, tlabels, S = data[cl]['X'], data[cl]['Y'], data[cl]['S']
            nid, rid, cid = data[cl]['label'],data[cl]['rsid'], data[cl]['csid']
            #timages = timages.to(device=device, dtype=torch.float32)
            timages= Variable(timages.cuda())
            si = Variable(S[0].cuda())
            with torch.no_grad():
                toutputs = model(timages.double(),si.double())
                predicted = toutputs.cpu().data
        
            predicted = predicted.squeeze()
            predicted = predicted.numpy()
            tids = list(nid.numpy())
            rid = list(rid.numpy())
            cid = list(cid.numpy())
            
            map_pred_img_ensemble = aggregate(map_pred_img_ensemble,predicted, rid, cid, 
                                          tids, patch_size)
    
    fname = 'rmse_all_'+str(batch_size)+'.txt'
    error_file=open(dir_out+fname,'w')
    total_err =0
    Ypred =[]
    for l in range(0,len(id_test)):
        tid = id_test[l][0]
        targets = map_pred_img_ensemble[tid]
        #loss = np.mean((Xtest[l] - targets)**2)
        loss = rmse(targets, Ytest[l][0])
        total_err+=loss
        #print(tid,loss,Xtest[l][0].shape, targets.shape)
        string=str(tid)+','+str(loss.item())+'\n'
        error_file.write(string)
        Ypred.append(targets)
        
    print('total:',total_err/len(id_test))
    string='total,'+str(total_err)+'\n'
    error_file.write(string)
        
    error_file.close()
    
    return Xtest, Ytest, Ypred, id_test


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xtest, ytest, ypred, idtest = test_all(device,batch_size, patch_size, dir_out, dir_model,
                                   dir_base_data, dir_super_data,s_dim)

Plot all rmse error for mesh

In [ ]:
import numpy as np
result_dir= 'results/hypernet-concentric-microbatch/rmse_all_50.txt'
rmse={}
#rmse_test={}
#rmse_train={}
with open(result_dir) as f:
    for line in f:
        node, rmse_sc = line.split(',')
        if node!='total':
            node = int(node)
            rmse_sc = float(rmse_sc)
            rmse[node] = rmse_sc
        
        #num_lines+=1


In [ ]:
normed_rmse = list(rmse.values())
normed_rmse = np.array(normed_rmse)
#tmp = (normed_rmse-np.min(normed_rmse))/(np.max(normed_rmse)-np.min(normed_rmse))

print(normed_rmse.shape)

In [ ]:

import matplotlib.pyplot as plt
#import colormaps as cmaps
from matplotlib import colors
import matplotlib.tri as tri

print(r.shape, z.shape,normed_rmse.shape, nnodes)

plt.figure(figsize=[10,16])
trimesh = tri.Triangulation(r, z, conn)
plt.triplot(trimesh, alpha=0.3)
plt.xlabel('R[m]')
plt.ylabel('Z[m]')
plt.title('XGC mesh')
plt.axis('scaled')
plt.tight_layout()

#plt.set_cmap(cmaps.viridis)
#norm = colors.Normalize(np.min(normed_rmse), np.max(normed_rmse))
norm = colors.Normalize(0, 0.23)
nc = plt.cm.hot(norm(normed_rmse))
for i in range(0,len(normed_rmse)):
  #nc = plt.cm.viridis(norm(normed_rmse[i]))
  #print(i,r[i],z[i], nc[i])
  plt.plot(r[i], z[i], color =nc[i,:], marker = 'o', lw=2)
  #plt.text(r[i], z[i], i)

#sm = plt.cm.ScalarMappable(cmap=plt.cm.hot, norm=colors.Normalize(np.min(normed_rmse), np.max(normed_rmse)))
sm = plt.cm.ScalarMappable(cmap=plt.cm.hot, norm=colors.Normalize(0, 0.23))
plt.colorbar(sm)
plt.savefig('results/hypernet-concentric-microbatch/mesh_error_all_lr_.0001.png')
plt.show()
